In [1]:
from IPython.core.display import HTML, display
from string import Template
import pandas as pd
import json, random

In [9]:
!ls

bar_chart.html	BarChart.ipynb	d3.js  draft  googleplaystore.csv  v1_dict.json


In [2]:
HTML("<script src='https://d3js.org/d3.v4.js'></script>")

In [3]:
df = pd.read_csv("googleplaystore.csv")
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [4]:
col='Rating'
v1=df[col].value_counts().reset_index()
v1.head()

,index,Rating
0,4.4,1109
1,4.3,1076
2,4.5,1038
3,4.2,952
4,4.6,823


In [5]:
v1=v1.rename(columns={col:'count','index':col})
v1.head()

,Rating,count
0,4.4,1109
1,4.3,1076
2,4.5,1038
3,4.2,952
4,4.6,823


In [6]:
v1_dict = v1.to_dict(orient="records")
v1_dict[:5]

[{'Rating': 4.4, 'count': 1109},
 {'Rating': 4.3, 'count': 1076},
 {'Rating': 4.5, 'count': 1038},
 {'Rating': 4.2, 'count': 952},
 {'Rating': 4.6, 'count': 823}]

In [30]:
import json

with open("v1_dict.json","w") as fh:
    #print('graph[0]', list(map(dict, graph[0])))
    json.dump(v1_dict,fh)

In [11]:
js_text_template = Template('''
var margin = {top:20,right:20,bottom:30,left:50};
var width = 1000 - margin.left - margin.right;
var height = 500 - margin.top - margin.bottom;



var y = d3.scaleLinear().range([height,0]);
var x = d3.scaleBand().range([0, width]).padding(0.2);
//d3.scale.linear().range([0,width]);

var svg = d3.select("#$maid").append("svg")
.attr("width", width + margin.left+margin.left)
.attr("height", height + margin.top+margin.bottom)
.append("g")
.attr("transform","translate("+margin.left+","+margin.top+")")

var data = $python_data;
data.forEach(function(d) {
    d.Rating = +d.Rating;
    d.count = +d.count;
  });
  
  data = data.sort(function(a, b) {
      return a.Rating - b.Rating;
  });
  x.domain(data.map(function(d) { return d.Rating;}));
  y.domain([0,d3.max(data, function(d) {return d.count;})]);

  var xAxis = svg.append("g")
  .attr("transform","translate(0,"+height+")")
  .call(d3.axisBottom(x))
  .append("text")
  .attr("class","label")
  .attr("x",width)
  .attr("y",height)
  .style("text-anchor","end")
  .text("Rating");

  var yAxis = svg.append("g")
  .call(d3.axisLeft(y));
  
  svg.selectAll(".bar")
  .data(data)
  .enter()
  .append("rect")
  .attr("class","bar")
  .attr("x", (d)=>x(d.Rating))
  .attr("y", (d)=>y(d.count))
  .attr("width", x.bandwidth())

  .transition()
  .ease(d3.easeLinear)
  .duration(500)
  .delay((d,i)=>i*50)
  .attr("height", (d)=>height-y(d.count));

''');

In [12]:
html_template = Template('''
<script src='https://d3js.org/d3.v4.js'></script>
    <style>
    .dot{
        stroke: #000;
    }
    .legend{
        color:blue;
    }
    .bar {
       fill: green;
     }
    
    .axis path,
    .axis line{
        fill: none;
        shape-rendering:crispEdges;
    
    }
    </style>
    <div id="$maid"></div>
    <script>$js_text</script>
''')
chart_id = "barchart-id"
js_text = js_text_template.substitute({
    "python_data":json.dumps(v1_dict),
    "maid":chart_id
})

HTML(html_template.substitute({"maid":chart_id,"js_text":js_text}))